In [7]:
import torch
from torch import nn

##### Dense - Fully Connected (FC) Layer - nn.Linear()

```nn.Linear(in_features, out_features)```

is equivalent to 

$ y = xW^T+b $ shape [1]

- $ x = [3,4,5] $ shape [1x3]

- $ w = [w_1, w_2, w_3] $ shape [1x3]

- $ b = b $ shape [1]

```in_features = 3```
```out_features = 1```




In [33]:

# Intantiate the layer
layer = nn.Linear(in_features=3, out_features=1, bias=True)  # weight shape = [1×3], bias = [1]
print(f"w = {layer.weight}\n")  # [1 × 3] torch tensor randomly initialized
print(f"b = {layer.bias}\n")    # [1] torch tensor randomly initialized

# Use the layer and check result
x = torch.tensor([1.0, 2.0, 3.0]) # input shape = [3]
y = layer(x) # output shape = [1]
#y_manual = x @ layer.weight.T + layer.bias # output shape = [1]
y_manual = torch.matmul(x, layer.weight.T) + layer.bias # output shape = [1]

assert y == y_manual
print(f"y = {y}\n")
print(f"y_manual = {y_manual}\n")

w = Parameter containing:
tensor([[-0.3451,  0.5244,  0.4520]], requires_grad=True)

b = Parameter containing:
tensor([-0.3777], requires_grad=True)

y = tensor([1.6821], grad_fn=<ViewBackward0>)

y_manual = tensor([1.6821], grad_fn=<AddBackward0>)

